# `mlarena.utils.data_utils` Demo

This notebook serves as a demonstration of the various data cleaning and manipulation utilities available in the `mlarena.utils.data_utils` module. 

In [11]:
import mlarena.utils.data_utils as dut
import pandas as pd
import numpy as np

# 1. Transform Data Columns

It is common for a dataframe to have date columns stored as strings. This handy function `transform_date_cols` helps you transform them. 

- Flexible input handling: Works with either a single column or multiple columns
- Format customization: Supports any date format using standard Python strftime directives
    - %d: Day of the month as a zero-padded decimal (e.g., 25)
    - %m: Month as a zero-padded decimal number (e.g., 08)
    - %b: Abbreviated month name (e.g., Aug)
    - %Y: Four-digit year (e.g., 2024)
- Smart case handling: Automatically normalizes month abbreviations (like 'JAN', 'jan', 'Jan') when using %b format
- Type safety: Preserves existing datetime columns without unnecessary conversion


In [12]:
# Sample DataFrame with different date formats
df_test = pd.DataFrame({
    "date1": ["20240101", "20240215", "20240320"],
    "date2": ["25-08-2024", "15-09-2024", "01-10-2024"],
    "date3": ["25Aug2024", "15AUG2024", "01aug2024"],  # different cases
    "date4": ["20240801", "20240915", "20240311"],
    "not_a_date": [123, "abc", None]
})
print(df_test.dtypes)

date1         object
date2         object
date3         object
date4         object
not_a_date    object
dtype: object


In [13]:
# Apply the function 
df_result = dut.transform_date_cols(df_test, ["date1", "date4"], "%Y%m%d") # take a list
df_result = dut.transform_date_cols(df_result, "date2", "%d-%m-%Y") # take one column
df_result = dut.transform_date_cols(df_result, ["date3"], "%d%b%Y") # handle column with different cases automatically

# Display result
print(df_result.dtypes)
print(df_result)

date1         datetime64[ns]
date2         datetime64[ns]
date3         datetime64[ns]
date4         datetime64[ns]
not_a_date            object
dtype: object
       date1      date2      date3      date4 not_a_date
0 2024-01-01 2024-08-25 2024-08-25 2024-08-01        123
1 2024-02-15 2024-09-15 2024-08-15 2024-09-15        abc
2 2024-03-20 2024-10-01 2024-08-01 2024-03-11       None


# 2. Clean Dollar Columns
It is common for a dataframe to have dollar amount columns stored as strings with currency symbols and commas. The `clean_dollar_cols` function helps you transform these into numeric values.

- Flexible input handling: Works with either a single column or multiple columns
- Clean the column(s) off currency symbols and commas
- Type conversion: Converts the cleaned strings to float values for numerical analysis


In [14]:
df_dollars = pd.DataFrame({
    'price': ['$1,234.56', '$2,345.67', '$3,456.78'],
    'revenue': ['12,000', '', '$30,000'],
    'other': ['A', 'B', 'C']
})

print("Original DataFrame:")
print(df_dollars)
print("\nDtypes:")
print(df_dollars.dtypes)

df_cleaned = dut.clean_dollar_cols(df_dollars, ['price', 'revenue'])

print("\nCleaned DataFrame:")
print(df_cleaned)
print("\nDtypes:")
print(df_cleaned.dtypes)

Original DataFrame:
       price  revenue other
0  $1,234.56   12,000     A
1  $2,345.67              B
2  $3,456.78  $30,000     C

Dtypes:
price      object
revenue    object
other      object
dtype: object

Cleaned DataFrame:
     price  revenue other
0  1234.56  12000.0     A
1  2345.67      NaN     B
2  3456.78  30000.0     C

Dtypes:
price      float64
revenue    float64
other       object
dtype: object


# 3. Value Counts with Percent
The `value_counts_with_pct` function enhances pandas' built-in value_counts by adding percentage information alongside counts.

- Comprehensive view: Shows both raw counts and percentages in a single output
- Flexible NA handling: Option to include or exclude NA values from the analysis
- Clear formatting: Percentages are formatted with a specified number of decimal places
- Sorted results: Values are sorted by frequency for easy interpretation
- Useful for: Quick categorical data profiling, understanding class distributions, and reporting

In [15]:
df_categories = pd.DataFrame({
    'color': ['red', 'blue', 'red', 'green', 'blue', 'red', None],
    'size': ['S', 'M', 'L', 'M', 'S', 'L', 'M']
})

print("Value counts for 'color' (including NA):")
print(dut.value_counts_with_pct(df_categories, 'color'))

print("\nValue counts for 'color' (excluding NA):")
print(dut.value_counts_with_pct(df_categories, 'color', dropna=True))

print("\nValue counts for ['color', 'size']:")
print(dut.value_counts_with_pct(df_categories, ['color','size']))


Value counts for 'color' (including NA):
   color  count    pct
0    red      3  42.86
1   blue      2  28.57
2  green      1  14.29
3   None      1  14.29

Value counts for 'color' (excluding NA):
   color  count    pct
0    red      3  50.00
1   blue      2  33.33
2  green      1  16.67

Value counts for ['color', 'size']:
   color size  count    pct
0    red    L      2  28.57
1   blue    M      1  14.29
2   blue    S      1  14.29
3  green    M      1  14.29
4    red    S      1  14.29
5    NaN    M      1  14.29


# 4. Drop Fully Null Columns
The `drop_fully_null_cols` function is specifically designed to prevent issues with Databricks' `display()` function, which can break when encountering columns that are entirely null (as it cannot infer the schema).

- Prevents Databricks display errors: Removes columns that would cause schema inference issues
- Safe operation: Returns a new DataFrame without modifying the original
- Common usage: `drop_fully_null_cols(df).display()` in Databricks notebooks.

In [16]:
df_nulls = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': [np.nan, np.nan, np.nan],  # Fully null
    'col3': ['A', None, 'C'],
    'col4': [None, None, None]  # Fully null
})

print("Original DataFrame:")
print(df_nulls)

df_cleaned = dut.drop_fully_null_cols(df_nulls, verbose=True)

print("\nCleaned DataFrame:")
print(df_cleaned) 

Original DataFrame:
   col1  col2  col3  col4
0     1   NaN     A  None
1     2   NaN  None  None
2     3   NaN     C  None
🗑️ Dropped fully-null columns: ['col2', 'col4']

Cleaned DataFrame:
   col1  col3
0     1     A
1     2  None
2     3     C


# 5. Print Schema Alphabetically
The `print_schema_alphabetically` function is particularly useful when exploring very wide DataFrames with many columns. By sorting column names alphabetically, it makes it easier to:

- Quickly locate specific columns in large datasets
- Compare schemas between different DataFrames to identify missing or additional columns
- Maintain a consistent view of your data structure regardless of the original column order
- Simplify documentation and reporting of data structures

In [17]:
df = pd.DataFrame({
    'z_price': [100.5, 200.5, 300.5],
    'a_category': ['A', 'B', 'C'],
    'm_date': pd.date_range('2024-01-01', periods=3),
    'b_is_active': [True, False, True],
    'y_quantity': np.array([1, 2, 3], dtype='int32')
})

print("Original DataFrame:")
print(df)
print("\nSchema in alphabetical order:")
dut.print_schema_alphabetically(df)

Original DataFrame:
   z_price a_category     m_date  b_is_active  y_quantity
0    100.5          A 2024-01-01         True           1
1    200.5          B 2024-01-02        False           2
2    300.5          C 2024-01-03         True           3

Schema in alphabetical order:
a_category             object
b_is_active              bool
m_date         datetime64[ns]
y_quantity              int32
z_price               float64
dtype: object


# 6. Check Primary Key
The `is_primary_key` function helps verify if a column or combination of columns could serve as a primary key in a DataFrame.

A traditional primary key must satisfy two key requirements:
1. Uniqueness: Each combination of values must be unique across all rows
2. No null values: Primary key columns cannot contain null/missing values

However, in real-world data analysis, we often encounter datasets where potential key columns contain some missing values. This function takes a practical approach by:
1. Alerting you about any missing values in the potential key columns
2. Checking if the columns would form a unique identifier after removing rows with missing values

This function is useful for:
- Data quality assessment: Understanding the completeness and uniqueness of your key fields
- Database schema design: Identifying potential primary keys even in imperfect data
- ETL validation: Verifying key constraints while being aware of data quality issues
- Data integrity checks: Ensuring uniqueness for joins/merges after handling missing values

The function accepts either a single column name or a list of columns, making it flexible for checking both simple and composite keys.

In [18]:
# Create sample DataFrame with different primary key scenarios
df = pd.DataFrame({
    # Single column primary key
    'id': [1, 2, 3, 4, 5],
    
    # Column with duplicates
    'category': ['A', 'B', 'A', 'B', 'C'],
    
    # Date column with some duplicates
    'date': ['2024-01-01', '2024-01-01', '2024-01-02', '2024-01-02', '2024-01-03'],
    
    # Column with null values
    'code': ['X1', None, 'X3', 'X4', 'X5'],
    
    # Values column
    'value': [100, 200, 300, 400, 500]
})

# Test 1: Single column that is a primary key
print("\nTest 1: Single column primary key")
dut.is_primary_key(df, ['id'])  # Should return True

# Test 2: Single column that is not a primary key (has duplicates)
print("\nTest 2: Column with duplicates")
dut.is_primary_key(df, ['category'])  # Should return False

# Test 3: Multiple columns that together form a primary key
print("\nTest 3: Composite primary key")
dut.is_primary_key(df, ['category', 'date'])  # Should return True

# Test 4: Column with null values
print("\nTest 4: Column with null values")
dut.is_primary_key(df, ['code','date'])  # Should return True

# Test 5: Empty DataFrame
print("\nTest 5: Empty DataFrame")
empty_df = pd.DataFrame(columns=['id', 'value'])
dut.is_primary_key(empty_df, ['id'])  # Should return False

# Test 6: Non-existent column
print("\nTest 6: Non-existent column")
dut.is_primary_key(df, ['not_a_column'])  # Should return False


Test 1: Single column primary key
✅ There are no missing values in column 'id'.
ℹ️ Total row count after filtering out missings: 5
ℹ️ Unique row count after filtering out missings: 5
🔑 The column(s) 'id' form a primary key.

Test 2: Column with duplicates
✅ There are no missing values in column 'category'.
ℹ️ Total row count after filtering out missings: 5
ℹ️ Unique row count after filtering out missings: 3
❌ The column(s) 'category' do not form a primary key.

Test 3: Composite primary key
✅ There are no missing values in columns 'category', 'date'.
ℹ️ Total row count after filtering out missings: 5
ℹ️ Unique row count after filtering out missings: 5
🔑 The column(s) 'category', 'date' form a primary key.

Test 4: Column with null values
⚠️ There are 1 row(s) with missing values in column 'code'.
✅ There are no missing values in column 'date'.
ℹ️ Total row count after filtering out missings: 4
ℹ️ Unique row count after filtering out missings: 4
🔑 The column(s) 'code', 'date' form a pr

False

# 7. Select Existing Columns
The `select_existing_cols` function provides a safe way to select columns from a DataFrame, handling cases where some requested columns might not exist.

- Safe column selection: Returns only columns that exist in the DataFrame
- Case sensitivity options: Can match column names exactly or case-insensitively (default: case sensitive)
- Verbose mode: Optional detailed output about which columns were found/missing (default: not verbose)
- Useful for: Data pipeline robustness, handling dynamic column selections

In [19]:
# Create sample DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9],
    'Mixed_Case': [10, 11, 12]
})

print("Original DataFrame:")
print(df)

# Example 1: Basic usage, case_sensitive, non-verbose
print("\nExample 1: Select existing columns")
result1 = dut.select_existing_cols(df, ['A', 'C', 'D'])
print(result1)

# Example 2: Case-insensitive matching
print("\nExample 2: Case-insensitive matching")
result2 = dut.select_existing_cols(df, ['a', 'mixed_case'], case_sensitive=False, verbose=True)
print(result2)

# Example 3: Verbose output
print("\nExample 3: Verbose output with missing columns")
result3 = dut.select_existing_cols(df, ['A', 'Missing1', 'B', 'Missing2'], verbose=True)
print(result3)

Original DataFrame:
   A  B  C  Mixed_Case
0  1  4  7          10
1  2  5  8          11
2  3  6  9          12

Example 1: Select existing columns
   A  C
0  1  7
1  2  8
2  3  9

Example 2: Case-insensitive matching
✅ Columns found: ['A', 'Mixed_Case']
   A  Mixed_Case
0  1          10
1  2          11
2  3          12

Example 3: Verbose output with missing columns
✅ Columns found: ['A', 'B']
⚠️ Columns not found: ['Missing1', 'Missing2']
   A  B
0  1  4
1  2  5
2  3  6


# 8. Find Duplicate Rows
The `find_duplicates` function identifies and returns duplicate rows based on specified columns, making it easy to detect data quality issues or investigate duplicate records.

Key features:
- **Flexible column selection**: Works with single column or list of columns
- **Comprehensive output**: Returns full duplicate rows with count information  
- **NULL handling**: Automatically excludes rows with NULL values in key columns
- **Smart column ordering**: Places count and key columns first, followed by other columns
- **Useful for**: Data quality assessment, duplicate investigation, data cleaning validation

Common use cases:
- **Customer data**: Find duplicate customers based on email, phone, or name
- **Transaction analysis**: Identify potential duplicate transactions  
- **Data validation**: Check for unexpected duplicates before processing
- **Business logic**: Find entities that appear multiple times in key dimensions

In [20]:
# Create sample customer data with various duplicate scenarios
customer_data = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'email': ['alice@email.com', 'bob@email.com', 'alice@email.com', 'charlie@email.com', 
              'david@email.com', 'bob@email.com', None, 'eve@email.com', 'david@email.com'],
    'phone': ['555-0001', '555-0002', '555-0003', '555-0004', 
              '555-0001', '555-0002', '555-0007', '555-0008', '555-0001'],
    'name': ['Alice Smith', 'Bob Johnson', 'Alice Brown', 'Charlie Davis',
             'David Wilson', 'Bob Johnson', 'Frank Miller', 'Eve Taylor', 'David Wilson'],
    'registration_date': ['2024-01-01', '2024-01-05', '2024-01-10', '2024-01-15',
                         '2024-01-20', '2024-01-25', '2024-01-30', '2024-02-01', '2024-02-05']
})

print("Original customer data:")
print(customer_data)
print(f"\nTotal customers: {len(customer_data)}")


Original customer data:
   customer_id              email     phone           name registration_date
0            1    alice@email.com  555-0001    Alice Smith        2024-01-01
1            2      bob@email.com  555-0002    Bob Johnson        2024-01-05
2            3    alice@email.com  555-0003    Alice Brown        2024-01-10
3            4  charlie@email.com  555-0004  Charlie Davis        2024-01-15
4            5    david@email.com  555-0001   David Wilson        2024-01-20
5            6      bob@email.com  555-0002    Bob Johnson        2024-01-25
6            7               None  555-0007   Frank Miller        2024-01-30
7            8      eve@email.com  555-0008     Eve Taylor        2024-02-01
8            9    david@email.com  555-0001   David Wilson        2024-02-05

Total customers: 9


In [21]:
# Example 1: Find customers with duplicate emails
print("Example 1: Customers with duplicate emails")
email_duplicates = dut.find_duplicates(customer_data, ['email'])
print(email_duplicates)
print(f"\nFound {len(email_duplicates)} rows with duplicate emails")

# Example 2: Find customers with duplicate phone numbers  
print("\n" + "="*60)
print("Example 2: Customers with duplicate phone numbers")
phone_duplicates = dut.find_duplicates(customer_data, ['phone'])
print(phone_duplicates)
print(f"\nFound {len(phone_duplicates)} rows with duplicate phone numbers")


Example 1: Customers with duplicate emails
   count            email  customer_id     phone          name  \
0      2  alice@email.com            1  555-0001   Alice Smith   
1      2  alice@email.com            3  555-0003   Alice Brown   
2      2    bob@email.com            2  555-0002   Bob Johnson   
3      2    bob@email.com            6  555-0002   Bob Johnson   
4      2  david@email.com            5  555-0001  David Wilson   
5      2  david@email.com            9  555-0001  David Wilson   

  registration_date  
0        2024-01-01  
1        2024-01-10  
2        2024-01-05  
3        2024-01-25  
4        2024-01-20  
5        2024-02-05  

Found 6 rows with duplicate emails

Example 2: Customers with duplicate phone numbers
   count     phone  customer_id            email          name  \
0      3  555-0001            1  alice@email.com   Alice Smith   
1      3  555-0001            5  david@email.com  David Wilson   
2      3  555-0001            9  david@email.com  David

In [22]:
# Example 3: Find customers with duplicate names AND emails (composite key)
print("Example 3: Customers with duplicate name AND email combinations")
name_email_duplicates = dut.find_duplicates(customer_data, ['name', 'email'])
print(name_email_duplicates)
print(f"\nFound {len(name_email_duplicates)} rows with duplicate name+email combinations")

# Example 4: Business scenario - detecting potential duplicate customers
print("\n" + "="*60)
print("Example 4: Comprehensive duplicate customer analysis")
print("Finding customers who might be the same person using multiple criteria...")

# Check for same person with different emails but same phone
same_phone_diff_email = customer_data.groupby('phone')['email'].nunique()
suspicious_phones = same_phone_diff_email[same_phone_diff_email > 1].index
print(f"\nPhone numbers used with multiple emails: {list(suspicious_phones)}")

if len(suspicious_phones) > 0:
    print("Detailed view of customers with same phone but different emails:")
    suspicious_data = customer_data[customer_data['phone'].isin(suspicious_phones)]
    print(suspicious_data[['customer_id', 'phone', 'email', 'name']].sort_values('phone'))


Example 3: Customers with duplicate name AND email combinations
   count          name            email  customer_id     phone  \
0      2   Bob Johnson    bob@email.com            2  555-0002   
1      2   Bob Johnson    bob@email.com            6  555-0002   
2      2  David Wilson  david@email.com            5  555-0001   
3      2  David Wilson  david@email.com            9  555-0001   

  registration_date  
0        2024-01-05  
1        2024-01-25  
2        2024-01-20  
3        2024-02-05  

Found 4 rows with duplicate name+email combinations

Example 4: Comprehensive duplicate customer analysis
Finding customers who might be the same person using multiple criteria...

Phone numbers used with multiple emails: ['555-0001']
Detailed view of customers with same phone but different emails:
   customer_id     phone            email          name
0            1  555-0001  alice@email.com   Alice Smith
4            5  555-0001  david@email.com  David Wilson
8            9  555-0001  

In [23]:
# Example 5: Handling edge cases
print("Example 5: Edge cases and NULL handling")

# Create data with edge cases
edge_case_data = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'category': ['A', 'B', None, 'A', None],  # Has NULLs
    'value': [100, 200, 300, 100, 400]
})

print("Data with NULL values:")
print(edge_case_data)

# Find duplicates - NULLs will be excluded
print("\nFinding duplicates in 'category' column (NULLs excluded):")
category_dups = dut.find_duplicates(edge_case_data, ['category'])
print(category_dups)

# Find duplicates in 'value' column
print("\nFinding duplicates in 'value' column:")
value_dups = dut.find_duplicates(edge_case_data, ['value'])
print(value_dups)

# Test with no duplicates
unique_data = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Charlie']
})

print("\nTesting with data that has no duplicates:")
no_dups = dut.find_duplicates(unique_data, ['name'])
print(no_dups)
print(f"Shape: {no_dups.shape} - Returns empty DataFrame with proper column structure")


Example 5: Edge cases and NULL handling
Data with NULL values:
   id category  value
0   1        A    100
1   2        B    200
2   3     None    300
3   4        A    100
4   5     None    400

Finding duplicates in 'category' column (NULLs excluded):
   count category  id  value
0      2        A   1    100
1      2        A   4    100

Finding duplicates in 'value' column:
   count  value  id category
0      2    100   1        A
1      2    100   4        A

Testing with data that has no duplicates:
Empty DataFrame
Columns: [count, name, id]
Index: []
Shape: (0, 3) - Returns empty DataFrame with proper column structure


# 9. Smart Deduplication by Ranking
The `deduplicate_by_rank` function provides intelligent duplicate removal by keeping the "best" record from each group based on a ranking criterion. This is the perfect complement to `find_duplicates` - first investigate, then intelligently resolve.

Key features:
- **Intelligent selection**: Keeps the best-ranked record per group instead of arbitrary duplicate removal
- **Flexible ranking**: Ascending or descending order based on any column (date, score, priority, etc.)
- **Advanced tiebreaking**: Prefer records with non-missing values in specified columns
- **Multi-column grouping**: Support for composite keys (customer_id + product_id)
- **Business-focused**: Designed for real-world scenarios like customer data cleanup

This function is essential for:
- **Customer master data**: Keep the most recent or complete customer record
- **Transaction deduplication**: Keep highest value or most recent transaction
- **Product catalog cleanup**: Keep most complete product information
- **Data quality improvement**: Systematic duplicate resolution with business logic



In [24]:
# Create sample customer data with duplicates for deduplication demo
customer_records = pd.DataFrame({
    'customer_id': ['C001', 'C001', 'C001', 'C002', 'C002', 'C003', 'C003', 'C003'],
    'record_date': ['2024-01-01', '2024-01-15', '2024-01-10', '2024-01-05', '2024-01-20', 
                   '2024-01-25', '2024-01-30', '2024-01-28'],
    'profile_completeness': [0.3, 0.9, 0.6, 0.4, 0.8, 0.7, 0.5, 0.9],
    'email': ['old@email.com', 'new@email.com', 'temp@email.com', None, 'customer@email.com', 
              'test@email.com', None, 'verified@email.com'],
    'phone': [None, '555-0001', '555-0002', None, '555-0003', '555-0004', '555-0005', '555-0006'],
    'total_orders': [5, 12, 8, 2, 15, 20, 18, 25],
    'last_purchase_amount': [150.00, 320.50, 200.00, 75.00, 450.00, 180.00, 220.00, 380.00]
})

print("Original customer records with duplicates:")
print(customer_records)
print(f"\nTotal records: {len(customer_records)}")
print(f"Unique customers: {customer_records['customer_id'].nunique()}")

# First, let's see what duplicates we have
print("\nUsing find_duplicates to investigate:")
duplicates = dut.find_duplicates(customer_records, ['customer_id'])
print(duplicates[['count', 'customer_id', 'record_date', 'profile_completeness', 'email']])


Original customer records with duplicates:
  customer_id record_date  profile_completeness               email     phone  \
0        C001  2024-01-01                   0.3       old@email.com      None   
1        C001  2024-01-15                   0.9       new@email.com  555-0001   
2        C001  2024-01-10                   0.6      temp@email.com  555-0002   
3        C002  2024-01-05                   0.4                None      None   
4        C002  2024-01-20                   0.8  customer@email.com  555-0003   
5        C003  2024-01-25                   0.7      test@email.com  555-0004   
6        C003  2024-01-30                   0.5                None  555-0005   
7        C003  2024-01-28                   0.9  verified@email.com  555-0006   

   total_orders  last_purchase_amount  
0             5                 150.0  
1            12                 320.5  
2             8                 200.0  
3             2                  75.0  
4            15            

In [25]:
# Strategy 1: Keep most recent record per customer
print("Strategy 1: Keep most recent record per customer")
most_recent = dut.deduplicate_by_rank(
    customer_records, 
    id_cols='customer_id', 
    ranking_col='record_date', 
    ascending=False,  # False = keep latest date
    verbose=True
)
print("\nMost recent records:")
print(most_recent[['customer_id', 'record_date', 'profile_completeness', 'email', 'total_orders']])

print("\n" + "="*70)

# Strategy 2: Keep record with highest profile completeness
print("Strategy 2: Keep record with highest profile completeness")
most_complete = dut.deduplicate_by_rank(
    customer_records,
    id_cols='customer_id',
    ranking_col='profile_completeness',
    ascending=False,  # False = keep highest completeness
    verbose=True
)
print("\nMost complete profiles:")
print(most_complete[['customer_id', 'record_date', 'profile_completeness', 'email', 'total_orders']])


Strategy 1: Keep most recent record per customer
🔄 Deduplicating 8 rows by ['customer_id']
ℹ️ Found 3 unique groups
✅ Removed 5 duplicate rows
📊 Final dataset: 3 rows

Most recent records:
  customer_id record_date  profile_completeness               email  \
0        C001  2024-01-15                   0.9       new@email.com   
1        C002  2024-01-20                   0.8  customer@email.com   
2        C003  2024-01-30                   0.5                None   

   total_orders  
0            12  
1            15  
2            18  

Strategy 2: Keep record with highest profile completeness
🔄 Deduplicating 8 rows by ['customer_id']
ℹ️ Found 3 unique groups
✅ Removed 5 duplicate rows
📊 Final dataset: 3 rows

Most complete profiles:
  customer_id record_date  profile_completeness               email  \
0        C001  2024-01-15                   0.9       new@email.com   
1        C002  2024-01-20                   0.8  customer@email.com   
2        C003  2024-01-28              

In [26]:
# Strategy 3: Advanced - Keep most recent record but break ties by preferring records with email
print("Strategy 3: Most recent record with email preference as tiebreaker")

# Create data where we have ties in dates to demonstrate tiebreaker
tied_data = pd.DataFrame({
    'customer_id': ['C004', 'C004', 'C005', 'C005'],
    'record_date': ['2024-01-15', '2024-01-15', '2024-01-20', '2024-01-20'],  # Same dates
    'profile_completeness': [0.7, 0.8, 0.6, 0.9],
    'email': [None, 'customer@email.com', 'user@email.com', None],  # One has email, one doesn't
    'phone': ['555-0007', '555-0008', '555-0009', '555-0010']
})

print("Data with tied dates:")
print(tied_data)

# Without tiebreaker - arbitrary selection
without_tiebreaker = dut.deduplicate_by_rank(
    tied_data,
    id_cols='customer_id',
    ranking_col='record_date',
    ascending=False
)
print("\nWithout tiebreaker (might select any tied record):")
print(without_tiebreaker)

# With tiebreaker - prefer non-null email  
with_tiebreaker = dut.deduplicate_by_rank(
    tied_data,
    id_cols='customer_id', 
    ranking_col='record_date',
    ascending=False,
    tiebreaker_col='email'  # Prefer records with non-null email
)
print("\nWith email tiebreaker (prefers non-null email):")
print(with_tiebreaker)


Strategy 3: Most recent record with email preference as tiebreaker
Data with tied dates:
  customer_id record_date  profile_completeness               email     phone
0        C004  2024-01-15                   0.7                None  555-0007
1        C004  2024-01-15                   0.8  customer@email.com  555-0008
2        C005  2024-01-20                   0.6      user@email.com  555-0009
3        C005  2024-01-20                   0.9                None  555-0010

Without tiebreaker (might select any tied record):
  customer_id record_date  profile_completeness           email     phone
0        C004  2024-01-15                   0.7            None  555-0007
1        C005  2024-01-20                   0.6  user@email.com  555-0009

With email tiebreaker (prefers non-null email):
  customer_id record_date  profile_completeness               email     phone
0        C004  2024-01-15                   0.8  customer@email.com  555-0008
1        C005  2024-01-20                 

In [27]:
# Business Use Case: Transaction Analysis
print("Business Use Case: Transaction Deduplication")

# Create transaction data with potential duplicates
transactions = pd.DataFrame({
    'customer_id': ['C001', 'C001', 'C001', 'C002', 'C002', 'C003', 'C003'],
    'product_id': ['P001', 'P001', 'P002', 'P001', 'P001', 'P002', 'P002'],
    'transaction_date': ['2024-01-01', '2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04', 
                        '2024-01-05', '2024-01-05'],
    'amount': [100.00, 150.00, 200.00, 75.00, 125.00, 180.00, 220.00],
    'payment_method': ['Credit', 'Credit', 'Debit', 'Cash', 'Credit', 'Credit', 'Debit'],
    'store_location': ['Store A', 'Store A', 'Store B', 'Store A', 'Store B', 'Store A', 'Store A']
})

print("Transaction data with potential duplicates:")
print(transactions)

# Find duplicates first
print("\nFinding duplicate customer-product combinations:")
transaction_dups = dut.find_duplicates(transactions, ['customer_id', 'product_id'])
print(transaction_dups)

# Strategy: Keep highest amount transaction per customer-product combination
print("\nDeduplication Strategy: Keep highest amount per customer-product")
deduplicated_transactions = dut.deduplicate_by_rank(
    transactions,
    id_cols=['customer_id', 'product_id'],
    ranking_col='amount',
    ascending=False,  # Keep highest amount
    verbose=True
)

print("\nFinal deduplicated transactions:")
print(deduplicated_transactions)

print(f"\nSummary:")
print(f"Original transactions: {len(transactions)}")
print(f"After deduplication: {len(deduplicated_transactions)}")
print(f"Removed duplicates: {len(transactions) - len(deduplicated_transactions)}")


Business Use Case: Transaction Deduplication
Transaction data with potential duplicates:
  customer_id product_id transaction_date  amount payment_method  \
0        C001       P001       2024-01-01   100.0         Credit   
1        C001       P001       2024-01-01   150.0         Credit   
2        C001       P002       2024-01-02   200.0          Debit   
3        C002       P001       2024-01-03    75.0           Cash   
4        C002       P001       2024-01-04   125.0         Credit   
5        C003       P002       2024-01-05   180.0         Credit   
6        C003       P002       2024-01-05   220.0          Debit   

  store_location  
0        Store A  
1        Store A  
2        Store B  
3        Store A  
4        Store B  
5        Store A  
6        Store A  

Finding duplicate customer-product combinations:
   count customer_id product_id transaction_date  amount payment_method  \
0      2        C001       P001       2024-01-01   100.0         Credit   
1      2      

## 9.1 Complete Duplicate Management Workflow

Together, the three functions below provide a systematic "Discover → Investigate → Resolve" approach:

- Use `is_primary_key` to discover the existance of duplication issues
- Use `find_duplicates` to analyze duplicate patterns
- Use `deduplicate_by_rank` to intelligently resolve duplicates with business logic

In [28]:
# Sample customer data with intentional quality issues
customer_data = pd.DataFrame({
    'customer_id': ['C001', 'C002', 'C003', 'C001', 'C004', 'C002', 'C005', 'C003', 'C006'],
    'email': ['alice@email.com', 'bob@email.com', 'charlie@email.com', 'alice.updated@email.com', 
              'diana@email.com', 'bob@email.com', 'eve@email.com', 'charlie.new@email.com', 'frank@email.com'],
    'phone': ['555-0001', '555-0002', '555-0003', '555-0001', '555-0004', 
              '555-0005', '555-0006', '555-0003', '555-0007'],
    'registration_date': ['2024-01-01', '2024-01-15', '2024-02-01', '2024-02-15', '2024-03-01',
                         '2024-03-15', '2024-04-01', '2024-04-15', '2024-05-01'],
    'profile_completeness': [0.6, 0.8, 0.4, 0.9, 0.7, 0.8, 0.5, 0.8, 0.6],
    'total_orders': [5, 12, 3, 8, 15, 12, 2, 7, 10],
    'last_purchase_amount': [150.50, 320.00, 75.25, 280.75, 450.00, 320.00, 95.50, 180.00, 220.00],
    'account_status': ['Active', 'Premium', 'Basic', 'Premium', 'Premium', 'Premium', 'Basic', 'Active', 'Active']
})

print("Original Customer Data:")
print(customer_data)
print(f"\nDataset Info: {len(customer_data)} rows, {customer_data.shape[1]} columns")



Original Customer Data:
  customer_id                    email     phone registration_date  \
0        C001          alice@email.com  555-0001        2024-01-01   
1        C002            bob@email.com  555-0002        2024-01-15   
2        C003        charlie@email.com  555-0003        2024-02-01   
3        C001  alice.updated@email.com  555-0001        2024-02-15   
4        C004          diana@email.com  555-0004        2024-03-01   
5        C002            bob@email.com  555-0005        2024-03-15   
6        C005            eve@email.com  555-0006        2024-04-01   
7        C003    charlie.new@email.com  555-0003        2024-04-15   
8        C006          frank@email.com  555-0007        2024-05-01   

   profile_completeness  total_orders  last_purchase_amount account_status  
0                   0.6             5                150.50         Active  
1                   0.8            12                320.00        Premium  
2                   0.4             3       

### STEP 1: DISCOVER - Check for Primary Key Issues

In [29]:
# Check if customer_id is a proper primary key
print("\n🔍 Checking if 'customer_id' can serve as primary key...")
is_unique = dut.is_primary_key(customer_data, 'customer_id', verbose=True)

if not is_unique:
    print("\n⚠️  PRIMARY KEY ISSUE DETECTED!")
    print("   Multiple records exist for the same customer_id")
    print("   → Need to investigate and resolve duplicates\n")
else:
    print("\n✅ No primary key issues detected")


🔍 Checking if 'customer_id' can serve as primary key...
✅ There are no missing values in column 'customer_id'.
ℹ️ Total row count after filtering out missings: 9
ℹ️ Unique row count after filtering out missings: 6
❌ The column(s) 'customer_id' do not form a primary key.

⚠️  PRIMARY KEY ISSUE DETECTED!
   Multiple records exist for the same customer_id
   → Need to investigate and resolve duplicates



### STEP 2: INVESTIGATE - Analyze Duplicate Patterns

In [30]:
# Find and analyze all duplicates
print("\n🔍 Finding duplicate customer records...")
duplicates = dut.find_duplicates(customer_data, ['customer_id'])

if len(duplicates) > 0:
    print(f"\n📊 DUPLICATE ANALYSIS RESULTS:")
    print(f"   • Found {len(duplicates)} duplicate rows")
    print(f"   • Affecting {duplicates['customer_id'].nunique()} customers")
    
    print(f"\n📋 Detailed duplicate records:")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    print(duplicates[['count', 'customer_id', 'email', 'registration_date', 'profile_completeness', 'total_orders']])
    
    # Analyze patterns
    print(f"\n🔍 DUPLICATE PATTERNS:")
    duplicate_customers = duplicates['customer_id'].unique()
    for customer in duplicate_customers:
        customer_records = duplicates[duplicates['customer_id'] == customer]
        print(f"\n   Customer {customer}:")
        print(f"   • {len(customer_records)} duplicate records")
        print(f"   • Registration dates: {customer_records['registration_date'].tolist()}")
        print(f"   • Profile completeness: {customer_records['profile_completeness'].tolist()}")
        print(f"   • Email addresses: {customer_records['email'].tolist()}")
else:
    print("✅ No duplicates found")



🔍 Finding duplicate customer records...

📊 DUPLICATE ANALYSIS RESULTS:
   • Found 6 duplicate rows
   • Affecting 3 customers

📋 Detailed duplicate records:
   count customer_id                    email registration_date  \
0      2        C001          alice@email.com        2024-01-01   
1      2        C001  alice.updated@email.com        2024-02-15   
2      2        C002            bob@email.com        2024-01-15   
3      2        C002            bob@email.com        2024-03-15   
4      2        C003        charlie@email.com        2024-02-01   
5      2        C003    charlie.new@email.com        2024-04-15   

   profile_completeness  total_orders  
0                   0.6             5  
1                   0.9             8  
2                   0.8            12  
3                   0.8            12  
4                   0.4             3  
5                   0.8             7  

🔍 DUPLICATE PATTERNS:

   Customer C001:
   • 2 duplicate records
   • Registration dates: 

### STEP 3: RESOLVE - Intelligently Remove Duplicates

In [31]:
print("\n🎯 DEDUPLICATION STRATEGY:")
print("   • Keep most recent registration (latest date)")
print("   • Break ties by preferring higher profile completeness")
print("   • Ensure one record per customer")

# Apply intelligent deduplication
cleaned_data = dut.deduplicate_by_rank(
    customer_data,
    id_cols='customer_id',
    ranking_col='registration_date',
    ascending=False,  # Keep most recent
    tiebreaker_col='email',  # Prefer non-null emails in case of ties
    verbose=True
)

print(f"\n📊 DEDUPLICATION RESULTS:")
print(f"   • Original records: {len(customer_data)}")
print(f"   • After deduplication: {len(cleaned_data)}")
print(f"   • Records removed: {len(customer_data) - len(cleaned_data)}")

print(f"\n✅ CLEANED CUSTOMER DATA:")
print(cleaned_data[['customer_id', 'email', 'registration_date', 'profile_completeness', 'total_orders', 'account_status']])




🎯 DEDUPLICATION STRATEGY:
   • Keep most recent registration (latest date)
   • Break ties by preferring higher profile completeness
   • Ensure one record per customer
🔄 Deduplicating 9 rows by ['customer_id']
ℹ️ Found 6 unique groups
✅ Removed 3 duplicate rows
📊 Final dataset: 6 rows

📊 DEDUPLICATION RESULTS:
   • Original records: 9
   • After deduplication: 6
   • Records removed: 3

✅ CLEANED CUSTOMER DATA:
  customer_id                    email registration_date  \
0        C001  alice.updated@email.com        2024-02-15   
1        C002            bob@email.com        2024-03-15   
2        C003    charlie.new@email.com        2024-04-15   
3        C004          diana@email.com        2024-03-01   
4        C005            eve@email.com        2024-04-01   
5        C006          frank@email.com        2024-05-01   

   profile_completeness  total_orders account_status  
0                   0.9             8        Premium  
1                   0.8            12        Premium

### STEP 4: VERIFY - Confirm Data Quality

In [32]:
# Verify the cleaned data
print("\n🔍 Verifying cleaned data quality...")
is_clean = dut.is_primary_key(cleaned_data, 'customer_id', verbose=True)

if is_clean:
    print("\n🎉 SUCCESS! Data quality issues resolved:")
    print("   ✅ customer_id is now a proper primary key")
    print("   ✅ No duplicate customers remain")
    print("   ✅ Most recent and complete records preserved")
    print("   ✅ Data ready for ML pipeline processing")

print("\n" + "="*70)
print("WORKFLOW SUMMARY")
print("="*70)

print("\n📋 COMPLETE DUPLICATE MANAGEMENT WORKFLOW:")
print("   1. 🔍 DISCOVER  → Used is_primary_key() to detect duplication issues")
print("   2. 📊 INVESTIGATE → Used find_duplicates() to analyze patterns") 
print("   3. 🎯 RESOLVE   → Used deduplicate_by_rank() to intelligently clean data")
print("   4. ✅ VERIFY    → Confirmed data quality with is_primary_key()")

print(f"\n🎯 BUSINESS IMPACT:")
print(f"   • Reduced dataset from {len(customer_data)} to {len(cleaned_data)} records")
print(f"   • Preserved most recent and complete customer information")
print(f"   • Eliminated duplicate customers for accurate analytics")
print(f"   • Ensured data quality for downstream ML processes")

print(f"\n💡 KEY BENEFITS:")
print(f"   • Systematic approach to data quality management")
print(f"   • Business logic embedded in deduplication strategy")
print(f"   • Transparent process with detailed reporting")
print(f"   • Production-ready cleaned dataset")


🔍 Verifying cleaned data quality...
✅ There are no missing values in column 'customer_id'.
ℹ️ Total row count after filtering out missings: 6
ℹ️ Unique row count after filtering out missings: 6
🔑 The column(s) 'customer_id' form a primary key.

🎉 SUCCESS! Data quality issues resolved:
   ✅ customer_id is now a proper primary key
   ✅ No duplicate customers remain
   ✅ Most recent and complete records preserved
   ✅ Data ready for ML pipeline processing

WORKFLOW SUMMARY

📋 COMPLETE DUPLICATE MANAGEMENT WORKFLOW:
   1. 🔍 DISCOVER  → Used is_primary_key() to detect duplication issues
   2. 📊 INVESTIGATE → Used find_duplicates() to analyze patterns
   3. 🎯 RESOLVE   → Used deduplicate_by_rank() to intelligently clean data
   4. ✅ VERIFY    → Confirmed data quality with is_primary_key()

🎯 BUSINESS IMPACT:
   • Reduced dataset from 9 to 6 records
   • Preserved most recent and complete customer information
   • Eliminated duplicate customers for accurate analytics
   • Ensured data qualit